# データサイエンス100本ノック（構造化データ加工編） - R

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあればinstall.packages()で適宜インストールしてください
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
require("RPostgreSQL")
require("tidyr")
require("dplyr")
require("stringr")
require("caret")
require("lubridate")
require("rsample")
require("recipes")
require("themis")

host <- "db"
port <- Sys.getenv()["PG_PORT"]
dbname <- Sys.getenv()["PG_DATABASE"]
user <- Sys.getenv()["PG_USER"]
password <- Sys.getenv()["PG_PASSWORD"]

con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
df_customer <- dbGetQuery(con,"SELECT * FROM customer")
df_category <- dbGetQuery(con,"SELECT * FROM category")
df_product <- dbGetQuery(con,"SELECT * FROM product")
df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
df_store <- dbGetQuery(con,"SELECT * FROM store")
df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

Loading required package: RPostgreSQL

Loading required package: DBI

Loading required package: tidyr

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: stringr

Loading required package: caret

Loading required package: ggplot2

Loading required package: lattice

Loading required package: lubridate


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading required package: rsample

Loading required package: recipes


Attaching package: ‘recipes’


The following object is masked from ‘package:stringr’:

    fixed


The following object is masked from ‘package:stats’:

    step


Loading required package: themis



# 演習問題

---
> R-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [2]:
head(df_receipt, n = 10)

sales_ymd sales_epoch store_cd receipt_no receipt_sub_no customer_id   
1  20181103  1541203200  S14006    112       1              CS006214000001
2  20181118  1542499200  S13008   1132       2              CS008415000097
3  20170712  1499817600  S14028   1102       1              CS028414000014
4  20190205  1549324800  S14042   1132       1              ZZ000000000000
5  20180821  1534809600  S14025   1102       2              CS025415000050
6  20190605  1559692800  S13003   1112       1              CS003515000195
7  20181205  1543968000  S14024   1102       2              CS024514000042
8  20190922  1569110400  S14040   1102       1              CS040415000178
9  20170504  1493856000  S13020   1112       2              ZZ000000000000
10 20191010  1570665600  S14027   1102       1              CS027514000015
   product_cd quantity amount
1  P070305012 1        158   
2  P070701017 1         81   
3  P060101005 1        170   
4  P050301001 1         25   
5  P060102007 1         90   
6  P050102002 1        138   
7  P080101005 1         30   
8  P070501004 1        128   
9  P071302010 1        770   
10 P071101003 1        680

---
> R-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [3]:
head(df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")], n = 10)

sales_ymd customer_id    product_cd amount
1  20181103  CS006214000001 P070305012 158   
2  20181118  CS008415000097 P070701017  81   
3  20170712  CS028414000014 P060101005 170   
4  20190205  ZZ000000000000 P050301001  25   
5  20180821  CS025415000050 P060102007  90   
6  20190605  CS003515000195 P050102002 138   
7  20181205  CS024514000042 P080101005  30   
8  20190922  CS040415000178 P070501004 128   
9  20170504  ZZ000000000000 P071302010 770   
10 20191010  CS027514000015 P071101003 680

---
> R-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更しながら抽出すること。

In [4]:
head(rename(df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")], 
            sales_date = sales_ymd), n = 10)

sales_date customer_id    product_cd amount
1  20181103   CS006214000001 P070305012 158   
2  20181118   CS008415000097 P070701017  81   
3  20170712   CS028414000014 P060101005 170   
4  20190205   ZZ000000000000 P050301001  25   
5  20180821   CS025415000050 P060102007  90   
6  20190605   CS003515000195 P050102002 138   
7  20181205   CS024514000042 P080101005  30   
8  20190922   CS040415000178 P070501004 128   
9  20170504   ZZ000000000000 P071302010 770   
10 20191010   CS027514000015 P071101003 680

---
> R-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [5]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001")

sales_ymd customer_id    product_cd amount
1  20180911  CS018205000001 P071401012 2200  
2  20180414  CS018205000001 P060104007  600  
3  20170614  CS018205000001 P050206001  990  
4  20170614  CS018205000001 P060702015  108  
5  20190216  CS018205000001 P071005024  102  
6  20180414  CS018205000001 P071101002  278  
7  20190226  CS018205000001 P070902035  168  
8  20190924  CS018205000001 P060805001  495  
9  20190226  CS018205000001 P071401020 2200  
10 20180911  CS018205000001 P071401005 1100  
11 20190216  CS018205000001 P040101002  218  
12 20190924  CS018205000001 P091503001  280

---
> R-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [6]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001" & amount >= 1000)

sales_ymd customer_id    product_cd amount
1 20180911  CS018205000001 P071401012 2200  
2 20190226  CS018205000001 P071401020 2200  
3 20180911  CS018205000001 P071401005 1100

---
> R-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [7]:
df_receipt[c("sales_ymd", "customer_id", 
             "product_cd", "quantity", "amount")] %>%
    filter(customer_id == "CS018205000001" & (amount >= 1000 | quantity >= 5))

sales_ymd customer_id    product_cd quantity amount
1 20180911  CS018205000001 P071401012 1        2200  
2 20180414  CS018205000001 P060104007 6         600  
3 20170614  CS018205000001 P050206001 5         990  
4 20190226  CS018205000001 P071401020 1        2200  
5 20180911  CS018205000001 P071401005 1        1100

---
> R-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [8]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001" & between(amount, 1000, 2000))

sales_ymd customer_id    product_cd amount
1 20180911  CS018205000001 P071401005 1100

---
> R-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [9]:
df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")] %>%
    filter(customer_id == "CS018205000001" &
           product_cd != "P071401019")

sales_ymd customer_id    product_cd amount
1  20180911  CS018205000001 P071401012 2200  
2  20180414  CS018205000001 P060104007  600  
3  20170614  CS018205000001 P050206001  990  
4  20170614  CS018205000001 P060702015  108  
5  20190216  CS018205000001 P071005024  102  
6  20180414  CS018205000001 P071101002  278  
7  20190226  CS018205000001 P070902035  168  
8  20190924  CS018205000001 P060805001  495  
9  20190226  CS018205000001 P071401020 2200  
10 20180911  CS018205000001 P071401005 1100  
11 20190216  CS018205000001 P040101002  218  
12 20190924  CS018205000001 P091503001  280

---
> R-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store %>% filter(!(prefecture_cd == "13" | floor_area > 900))`

In [10]:
df_store %>% filter(prefecture_cd != "13" & floor_area <= 900)

store_cd store_name prefecture_cd prefecture
1 S14046   北山田店   14            神奈川県  
2 S14011   日吉本町店 14            神奈川県  
3 S12013   習志野店   12            千葉県    
  address                           
1 神奈川県横浜市都筑区北山田一丁目  
2 神奈川県横浜市港北区日吉本町四丁目
3 千葉県習志野市芝園一丁目          
  address_kana                                                 tel_no      
1 カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ         045-123-4049
2 カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ 045-123-4033
3 チバケンナラシノシシバゾノイッチョウメ                       047-123-4002
  longitude latitude floor_area
1 139.5916  35.56189 831       
2 139.6316  35.54655 890       
3 140.0220  35.66122 808

---
> R-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [11]:
head(df_store %>% filter(startsWith(store_cd, "S14")), n = 10)

store_cd store_name   prefecture_cd prefecture
1  S14010   菊名店       14            神奈川県  
2  S14033   阿久和店     14            神奈川県  
3  S14036   相模原中央店 14            神奈川県  
4  S14040   長津田店     14            神奈川県  
5  S14050   阿久和西店   14            神奈川県  
6  S14028   二ツ橋店     14            神奈川県  
7  S14012   本牧和田店   14            神奈川県  
8  S14046   北山田店     14            神奈川県  
9  S14022   逗子店       14            神奈川県  
10 S14011   日吉本町店   14            神奈川県  
   address                               
1  神奈川県横浜市港北区菊名一丁目        
2  神奈川県横浜市瀬谷区阿久和西一丁目    
3  神奈川県相模原市中央二丁目            
4  神奈川県横浜市緑区長津田みなみ台五丁目
5  神奈川県横浜市瀬谷区阿久和西一丁目    
6  神奈川県横浜市瀬谷区二ツ橋町          
7  神奈川県横浜市中区本牧和田            
8  神奈川県横浜市都筑区北山田一丁目      
9  神奈川県逗子市逗子一丁目              
10 神奈川県横浜市港北区日吉本町四丁目    
   address_kana                                                 tel_no      
1  カナガワケンヨコハマシコウホククキクナイッチョウメ           045-123-4032
2  カナガワケンヨコハマシセヤクアクワニシイッチョウメ           045-123-4043
3  カナガワケンサガミハラシチュウオウニチョウメ                 042-123-4045
4  カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ   045-123-4046
5  カナガワケンヨコハマシセヤクアクワニシイッチョウメ           045-123-4053
6  カナガワケンヨコハマシセヤクフタツバシチョウ                 045-123-4042
7  カナガワケンヨコハマシナカクホンモクワダ                     045-123-4034
8  カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ         045-123-4049
9  カナガワケンズシシズシイッチョウメ                           046-123-4036
10 カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ 045-123-4033
   longitude latitude floor_area
1  139.6326  35.50049 1732      
2  139.4961  35.45918 1495      
3  139.3716  35.57327 1679      
4  139.4994  35.52398 1548      
5  139.4961  35.45918 1830      
6  139.4963  35.46304 1574      
7  139.6582  35.42156 1341      
8  139.5916  35.56189  831      
9  139.5789  35.29642 1838      
10 139.6316  35.54655  890

---
> R-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [12]:
head(df_customer %>% filter(endsWith(customer_id, "1")), n = 10)

customer_id    customer_name gender_cd gender birth_day  age postal_cd
1  CS037613000071 六角 雅彦     9         不明   1952-04-01 66  136-0076 
2  CS028811000001 堀井 かおり   1         女性   1933-03-27 86  245-0016 
3  CS040412000191 川井 郁恵     1         女性   1977-01-05 42  226-0021 
4  CS028314000011 小菅 あおい   1         女性   1983-11-26 35  246-0038 
5  CS039212000051 藤島 恵梨香   1         女性   1997-02-03 22  166-0001 
6  CS015412000111 松居 奈月     1         女性   1972-10-04 46  136-0071 
7  CS004702000041 野島 洋       0         男性   1943-08-24 75  176-0022 
8  CS041515000001 栗田 千夏     1         女性   1967-01-02 52  206-0001 
9  CS029313000221 北条 ひかり   1         女性   1987-06-19 31  279-0011 
10 CS034312000071 望月 奈央     1         女性   1980-09-20 38  213-0026 
   address                              application_store_cd application_date
1  東京都江東区南砂**********           S13037               20150414        
2  神奈川県横浜市泉区和泉町**********   S14028               20160115        
3  神奈川県横浜市緑区北八朔町********** S14040               20151101        
4  神奈川県横浜市瀬谷区宮沢**********   S14028               20151123        
5  東京都杉並区阿佐谷北**********       S13039               20171121        
6  東京都江東区亀戸**********           S13015               20150629        
7  東京都練馬区向山**********           S13004               20170218        
8  東京都多摩市和田**********           S13041               20160422        
9  千葉県浦安市美浜**********           S12029               20180810        
10 神奈川県川崎市高津区久末**********   S14034               20160106        
   status_cd   
1  0-00000000-0
2  0-00000000-0
3  1-20091025-4
4  1-20080426-5
5  1-20100215-4
6  0-00000000-0
7  0-00000000-0
8  E-20100803-F
9  0-00000000-0
10 0-00000000-0

---
> R-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [13]:
df_store %>% filter(grepl("横浜市", address))

store_cd store_name prefecture_cd prefecture
1  S14010   菊名店     14            神奈川県  
2  S14033   阿久和店   14            神奈川県  
3  S14040   長津田店   14            神奈川県  
4  S14050   阿久和西店 14            神奈川県  
5  S14028   二ツ橋店   14            神奈川県  
6  S14012   本牧和田店 14            神奈川県  
7  S14046   北山田店   14            神奈川県  
8  S14011   日吉本町店 14            神奈川県  
9  S14048   中川中央店 14            神奈川県  
10 S14042   新山下店   14            神奈川県  
11 S14006   葛が谷店   14            神奈川県  
   address                               
1  神奈川県横浜市港北区菊名一丁目        
2  神奈川県横浜市瀬谷区阿久和西一丁目    
3  神奈川県横浜市緑区長津田みなみ台五丁目
4  神奈川県横浜市瀬谷区阿久和西一丁目    
5  神奈川県横浜市瀬谷区二ツ橋町          
6  神奈川県横浜市中区本牧和田            
7  神奈川県横浜市都筑区北山田一丁目      
8  神奈川県横浜市港北区日吉本町四丁目    
9  神奈川県横浜市都筑区中川中央二丁目    
10 神奈川県横浜市中区新山下二丁目        
11 神奈川県横浜市都筑区葛が谷            
   address_kana                                                 tel_no      
1  カナガワケンヨコハマシコウホククキクナイッチョウメ           045-123-4032
2  カナガワケンヨコハマシセヤクアクワニシイッチョウメ           045-123-4043
3  カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ   045-123-4046
4  カナガワケンヨコハマシセヤクアクワニシイッチョウメ           045-123-4053
5  カナガワケンヨコハマシセヤクフタツバシチョウ                 045-123-4042
6  カナガワケンヨコハマシナカクホンモクワダ                     045-123-4034
7  カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ         045-123-4049
8  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ 045-123-4033
9  カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ   045-123-4051
10 カナガワケンヨコハマシナカクシンヤマシタニチョウメ           045-123-4047
11 カナガワケンヨコハマシツヅキククズガヤ                       045-123-4031
   longitude latitude floor_area
1  139.6326  35.50049 1732      
2  139.4961  35.45918 1495      
3  139.4994  35.52398 1548      
4  139.4961  35.45918 1830      
5  139.4963  35.46304 1574      
6  139.6582  35.42156 1341      
7  139.5916  35.56189  831      
8  139.6316  35.54655  890      
9  139.5758  35.54912 1657      
10 139.6593  35.43894 1044      
11 139.5633  35.53573 1886

---
> R-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [14]:
head(df_customer %>% filter(grepl("^[A-F]", status_cd)), n = 10)

customer_id    customer_name gender_cd gender birth_day  age postal_cd
1  CS031415000172 宇多田 貴美子 1         女性   1976-10-04 42  151-0053 
2  CS015414000103 奥野 陽子     1         女性   1977-08-09 41  136-0073 
3  CS011215000048 芦田 沙耶     1         女性   1992-02-01 27  223-0062 
4  CS029415000023 梅田 里穂     1         女性   1976-01-17 43  279-0043 
5  CS035415000029 寺沢 真希     9         不明   1977-09-27 41  158-0096 
6  CS031415000106 宇野 由美子   1         女性   1970-02-26 49  151-0053 
7  CS029215000025 石倉 美帆     1         女性   1993-09-28 25  279-0022 
8  CS033605000005 猪股 雄太     0         男性   1955-12-05 63  246-0031 
9  CS033415000229 板垣 菜々美   1         女性   1977-11-07 41  246-0021 
10 CS008415000145 黒谷 麻緒     1         女性   1977-06-27 41  157-0067 
   address                                application_store_cd application_date
1  東京都渋谷区代々木**********           S13031               20150529        
2  東京都江東区北砂**********             S13015               20150722        
3  神奈川県横浜市港北区日吉本町********** S14011               20150228        
4  千葉県浦安市富士見**********           S12029               20150610        
5  東京都世田谷区玉川台**********         S13035               20141220        
6  東京都渋谷区代々木**********           S13031               20150201        
7  千葉県浦安市今川**********             S12029               20150708        
8  神奈川県横浜市瀬谷区瀬谷**********     S14033               20150425        
9  神奈川県横浜市瀬谷区二ツ橋町********** S14033               20150712        
10 東京都世田谷区喜多見**********         S13008               20150829        
   status_cd   
1  D-20100325-C
2  B-20100609-B
3  C-20100421-9
4  D-20100918-E
5  F-20101029-F
6  F-20100511-E
7  B-20100820-C
8  F-20100917-E
9  F-20100326-E
10 F-20100622-F

---
> R-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [15]:
head(df_customer %>% filter(grepl("[1-9]$", status_cd)), n = 10)

customer_id    customer_name gender_cd gender birth_day  age postal_cd
1  CS001215000145 田崎 美紀     1         女性   1995-03-29 24  144-0055 
2  CS033513000180 安斎 遥       1         女性   1962-07-11 56  241-0823 
3  CS011215000048 芦田 沙耶     1         女性   1992-02-01 27  223-0062 
4  CS040412000191 川井 郁恵     1         女性   1977-01-05 42  226-0021 
5  CS009315000023 皆川 文世     1         女性   1980-04-15 38  154-0012 
6  CS015315000033 福士 璃奈子   1         女性   1983-03-17 36  135-0043 
7  CS023513000066 神戸 そら     1         女性   1961-12-17 57  210-0005 
8  CS035513000134 市川 美帆     1         女性   1960-03-27 59  156-0053 
9  CS001515000263 高松 夏空     1         女性   1962-11-09 56  144-0051 
10 CS040314000027 鶴田 きみまろ 9         不明   1986-03-26 33  226-0027 
   address                                application_store_cd application_date
1  東京都大田区仲六郷**********           S13001               20170605        
2  神奈川県横浜市旭区善部町**********     S14033               20150728        
3  神奈川県横浜市港北区日吉本町********** S14011               20150228        
4  神奈川県横浜市緑区北八朔町**********   S14040               20151101        
5  東京都世田谷区駒沢**********           S13009               20150319        
6  東京都江東区塩浜**********             S13015               20141024        
7  神奈川県川崎市川崎区東田町**********   S14023               20150915        
8  東京都世田谷区桜**********             S13035               20150227        
9  東京都大田区西蒲田**********           S13001               20160812        
10 神奈川県横浜市緑区長津田**********     S14040               20150122        
   status_cd   
1  6-20090929-2
2  6-20080506-5
3  C-20100421-9
4  1-20091025-4
5  5-20080322-1
6  4-20080219-3
7  5-20100524-9
8  8-20100711-9
9  1-20100804-1
10 2-20080426-4

---
> R-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [16]:
head(df_customer %>% filter(grepl("^[A-F].*[1-9]$", status_cd)), n = 10)

customer_id    customer_name gender_cd gender birth_day  age postal_cd
1  CS011215000048 芦田 沙耶     1         女性   1992-02-01 27  223-0062 
2  CS022513000105 島村 貴美子   1         女性   1962-03-12 57  249-0002 
3  CS001515000096 水野 陽子     9         不明   1960-11-29 58  144-0053 
4  CS013615000053 西脇 季衣     1         女性   1953-10-18 65  261-0026 
5  CS020412000161 小宮 薫       1         女性   1974-05-21 44  174-0042 
6  CS001215000097 竹中 あさみ   1         女性   1990-07-25 28  146-0095 
7  CS035212000007 内村 恵梨香   1         女性   1990-12-04 28  152-0023 
8  CS002515000386 野田 コウ     1         女性   1963-05-30 55  185-0013 
9  CS001615000372 稲垣 寿々花   1         女性   1956-10-29 62  144-0035 
10 CS032512000121 松井 知世     1         女性   1962-09-04 56  210-0011 
   address                                application_store_cd application_date
1  神奈川県横浜市港北区日吉本町********** S14011               20150228        
2  神奈川県逗子市山の根**********         S14022               20150320        
3  東京都大田区蒲田本町**********         S13001               20150614        
4  千葉県千葉市美浜区幕張西**********     S12013               20150128        
5  東京都板橋区東坂下**********           S13020               20150822        
6  東京都大田区多摩川**********           S13001               20170315        
7  東京都目黒区八雲**********             S13035               20151013        
8  東京都国分寺市西恋ケ窪**********       S13002               20160410        
9  東京都大田区南蒲田**********           S13001               20170403        
10 神奈川県川崎市川崎区富士見**********   S13032               20150727        
   status_cd   
1  C-20100421-9
2  A-20091115-7
3  A-20100724-7
4  B-20100329-6
5  B-20081021-3
6  A-20100211-2
7  B-20101018-6
8  C-20100127-8
9  A-20100104-1
10 A-20100103-5

---
> R-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [17]:
df_store %>% filter(grepl("^[0-9]{3}-[0-9]{3}-[0-9]{4}$", tel_no))

store_cd store_name   prefecture_cd prefecture
1  S12014   千草台店     12            千葉県    
2  S13002   国分寺店     13            東京都    
3  S14010   菊名店       14            神奈川県  
4  S14033   阿久和店     14            神奈川県  
5  S14036   相模原中央店 14            神奈川県  
6  S14040   長津田店     14            神奈川県  
7  S14050   阿久和西店   14            神奈川県  
8  S13052   森野店       13            東京都    
9  S14028   二ツ橋店     14            神奈川県  
10 S14012   本牧和田店   14            神奈川県  
11 S14046   北山田店     14            神奈川県  
12 S14022   逗子店       14            神奈川県  
13 S14011   日吉本町店   14            神奈川県  
14 S13016   小金井店     13            東京都    
15 S14034   川崎野川店   14            神奈川県  
16 S14048   中川中央店   14            神奈川県  
17 S12007   佐倉店       12            千葉県    
18 S14026   辻堂西海岸店 14            神奈川県  
19 S13041   八王子店     13            東京都    
20 S14049   川崎大師店   14            神奈川県  
21 S14023   川崎店       14            神奈川県  
22 S13018   清瀬店       13            東京都    
23 S14027   南藤沢店     14            神奈川県  
24 S14021   伊勢原店     14            神奈川県  
25 S14047   相模原店     14            神奈川県  
26 S12013   習志野店     12            千葉県    
27 S14042   新山下店     14            神奈川県  
28 S12030   八幡店       12            千葉県    
29 S14025   大和店       14            神奈川県  
30 S14045   厚木店       14            神奈川県  
31 S12029   東野店       12            千葉県    
32 S12053   高洲店       12            千葉県    
33 S14024   三田店       14            神奈川県  
34 S14006   葛が谷店     14            神奈川県  
   address                               
1  千葉県千葉市稲毛区千草台一丁目        
2  東京都国分寺市本多二丁目              
3  神奈川県横浜市港北区菊名一丁目        
4  神奈川県横浜市瀬谷区阿久和西一丁目    
5  神奈川県相模原市中央二丁目            
6  神奈川県横浜市緑区長津田みなみ台五丁目
7  神奈川県横浜市瀬谷区阿久和西一丁目    
8  東京都町田市森野三丁目                
9  神奈川県横浜市瀬谷区二ツ橋町          
10 神奈川県横浜市中区本牧和田            
11 神奈川県横浜市都筑区北山田一丁目      
12 神奈川県逗子市逗子一丁目              
13 神奈川県横浜市港北区日吉本町四丁目    
14 東京都小金井市本町一丁目              
15 神奈川県川崎市宮前区野川              
16 神奈川県横浜市都筑区中川中央二丁目    
17 千葉県佐倉市上志津                    
18 神奈川県藤沢市辻堂西海岸二丁目        
19 東京都八王子市大塚                    
20 神奈川県川崎市川崎区中瀬三丁目        
21 神奈川県川崎市川崎区本町二丁目        
22 東京都清瀬市松山一丁目                
23 神奈川県藤沢市南藤沢                  
24 神奈川県伊勢原市伊勢原四丁目          
25 神奈川県相模原市千代田六丁目          
26 千葉県習志野市芝園一丁目              
27 神奈川県横浜市中区新山下二丁目        
28 千葉県市川市八幡三丁目                
29 神奈川県大和市下和田                  
30 神奈川県厚木市中町二丁目              
31 千葉県浦安市東野一丁目                
32 千葉県浦安市高洲五丁目                
33 神奈川県川崎市多摩区三田四丁目        
34 神奈川県横浜市都筑区葛が谷            
   address_kana                                                 tel_no      
1  チバケンチバシイナゲクチグサダイイッチョウメ                 043-123-4003
2  トウキョウトコクブンジシホンダニチョウメ                     042-123-4008
3  カナガワケンヨコハマシコウホククキクナイッチョウメ           045-123-4032
4  カナガワケンヨコハマシセヤクアクワニシイッチョウメ           045-123-4043
5  カナガワケンサガミハラシチュウオウニチョウメ                 042-123-4045
6  カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ   045-123-4046
7  カナガワケンヨコハマシセヤクアクワニシイッチョウメ           045-123-4053
8  トウキョウトマチダシモリノサンチョウメ                       042-123-4030
9  カナガワケンヨコハマシセヤクフタツバシチョウ                 045-123-4042
10 カナガワケンヨコハマシナカクホンモクワダ                     045-123-4034
11 カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ         045-123-4049
12 カナガワケンズシシズシイッチョウメ                           046-123-4036
13 カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ 045-123-4033
14 トウキョウトコガネイシホンチョウイッチョウメ                 042-123-4015
15 カナガワケンカワサキシミヤマエクノガワ                       044-123-4044
16 カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ   045-123-4051
17 チバケンサクラシカミシヅ                                     043-123-4001
18 カナガワケンフジサワシツジドウニシカイガンニチョウメ         046-123-4040
19 トウキョウトハチオウジシオオツカ                             042-123-4026
20 カナガワケンカワサキシカワサキクナカゼサンチョウメ           044-123-4052
21 カナガワケンカワサキシカワサキクホンチョウニチョウメ         044-123-4037
22 トウキョウトキヨセシマツヤマイッチョウメ                     042-123-4017
23 カナガワケンフジサワシミナミフジサワ                         046-123-4041
24 カナガワケンイセハラシイセハラヨンチョウメ                   046-123-4035
25 カナガワケンサガミハラシチヨダロクチョウメ                   042-123-4050
26 チバケンナラシノシシバゾノイッチョウメ                       047-12

---
> R-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [18]:
head(
    df_customer[order(df_customer$birth_day), ],
    n = 10
)

customer_id    customer_name gender_cd gender birth_day  age postal_cd
18818 CS003813000014 村山 菜々美   1         女性   1928-11-26 90  182-0007 
12329 CS026813000004 吉村 朝陽     1         女性   1928-12-14 90  251-0043 
15683 CS018811000003 熊沢 美里     1         女性   1929-01-07 90  204-0004 
15303 CS027803000004 内村 拓郎     0         男性   1929-01-12 90  251-0031 
1682  CS013801000003 天野 拓郎     0         男性   1929-01-15 90  274-0824 
7512  CS001814000022 鶴田 里穂     1         女性   1929-01-28 90  144-0045 
2379  CS016815000002 山元 美紀     1         女性   1929-02-22 90  184-0005 
4681  CS009815000003 中田 里穂     1         女性   1929-04-08 89  154-0014 
6306  CS012813000013 宇野 南朋     1         女性   1929-04-09 89  231-0806 
16071 CS005813000015 金谷 恵梨香   1         女性   1929-04-09 89  165-0032 
      address                            application_store_cd application_date
18818 東京都調布市菊野台**********       S13003               20160214        
12329 神奈川県藤沢市辻堂元町**********   S14026               20150723        
15683 東京都清瀬市野塩**********         S13018               20150403        
15303 神奈川県藤沢市鵠沼藤が谷********** S14027               20151227        
1682  千葉県船橋市前原東**********       S12013               20160120        
7512  東京都大田区南六郷**********       S13001               20161012        
2379  東京都小金井市桜町**********       S13016               20150629        
4681  東京都世田谷区新町**********       S13009               20150421        
6306  神奈川県横浜市中区本牧町********** S14012               20150712        
16071 東京都中野区鷺宮**********         S13005               20150506        
      status_cd   
18818 0-00000000-0
12329 0-00000000-0
15683 0-00000000-0
15303 0-00000000-0
1682  0-00000000-0
7512  A-20090415-7
2379  C-20090923-C
4681  D-20091021-E
6306  0-00000000-0
16071 0-00000000-0

---
> R-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [19]:
head(
    df_customer[order(df_customer$birth_day,
                      decreasing = TRUE), ],
    n = 10
)

customer_id    customer_name gender_cd gender birth_day  age postal_cd
15640 CS035114000004 大村 美里     1         女性   2007-11-25 11  156-0053 
7469  CS022103000002 福山 はじめ   9         不明   2007-10-02 11  249-0006 
10746 CS002113000009 柴田 真悠子   1         女性   2007-09-17 11  184-0014 
19812 CS004115000014 松井 京子     1         女性   2007-08-09 11  165-0031 
7040  CS002114000010 山内 遥       1         女性   2007-06-03 11  184-0015 
3671  CS025115000002 小柳 夏希     1         女性   2007-04-18 11  245-0018 
12494 CS002113000025 広末 まなみ   1         女性   2007-03-30 12  184-0015 
15978 CS033112000003 長野 美紀     1         女性   2007-03-22 12  245-0051 
5717  CS007115000006 福岡 瞬       1         女性   2007-03-10 12  285-0845 
15098 CS014113000008 矢口 莉緒     1         女性   2007-03-05 12  260-0041 
      address                              application_store_cd
15640 東京都世田谷区桜**********           S13035              
7469  神奈川県逗子市逗子**********         S14022              
10746 東京都小金井市貫井南町**********     S13002              
19812 東京都中野区上鷺宮**********         S13004              
7040  東京都小金井市貫井北町**********     S13002              
3671  神奈川県横浜市泉区上飯田町********** S14025              
12494 東京都小金井市貫井北町**********     S13002              
15978 神奈川県横浜市戸塚区名瀬町********** S14033              
5717  千葉県佐倉市西志津**********         S12007              
15098 千葉県千葉市中央区東千葉**********   S12014              
      application_date status_cd   
15640 20150619         6-20091205-6
7469  20160909         0-00000000-0
10746 20160304         0-00000000-0
19812 20161120         1-20081231-1
7040  20160920         6-20100510-1
3671  20160116         D-20100913-D
12494 20171030         0-00000000-0
15978 20150606         0-00000000-0
5717  20151118         F-20101016-F
15098 20150622         3-20091108-6

---
> R-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [20]:
df_receipt[c("customer_id", "amount")] %>%
    mutate(ranking = min_rank(desc(amount))) %>%
    arrange(ranking) %>%
    slice(1:10)

customer_id    amount ranking
1  CS011415000006 10925  1      
2  ZZ000000000000  6800  2      
3  CS028605000002  5780  3      
4  CS015515000034  5480  4      
5  ZZ000000000000  5480  4      
6  ZZ000000000000  5480  4      
7  ZZ000000000000  5440  7      
8  CS021515000089  5440  7      
9  CS015515000083  5280  9      
10 CS017414000114  5280  9

---
> R-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [21]:
df_receipt[c("customer_id", "amount")] %>%
    mutate(ranking = row_number(desc(amount))) %>%
    arrange(ranking) %>%
    slice(1:10) 

customer_id    amount ranking
1  CS011415000006 10925   1     
2  ZZ000000000000  6800   2     
3  CS028605000002  5780   3     
4  CS015515000034  5480   4     
5  ZZ000000000000  5480   5     
6  ZZ000000000000  5480   6     
7  ZZ000000000000  5440   7     
8  CS021515000089  5440   8     
9  CS015515000083  5280   9     
10 CS017414000114  5280  10

---
> R-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [22]:
nrow(df_receipt)

[1] 104681

---
> R-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [23]:
length(unique(df_receipt$customer_id))

[1] 8307

---
> R-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [24]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(amount = sum(amount),
              quantity = sum(quantity), .groups = "drop")

store_cd amount quantity
1  S12007   638761 2099    
2  S12013   787513 2425    
3  S12014   725167 2358    
4  S12029   794741 2555    
5  S12030   684402 2403    
6  S13001   811936 2347    
7  S13002   727821 2340    
8  S13003   764294 2197    
9  S13004   779373 2390    
10 S13005   629876 2004    
11 S13008   809288 2491    
12 S13009   808870 2486    
13 S13015   780873 2248    
14 S13016   793773 2432    
15 S13017   748221 2376    
16 S13018   790535 2562    
17 S13019   827833 2541    
18 S13020   796383 2383    
19 S13031   705968 2336    
20 S13032   790501 2491    
21 S13035   715869 2219    
22 S13037   693087 2344    
23 S13038   708884 2337    
24 S13039   611888 1981    
25 S13041   728266 2233    
26 S13043   587895 1881    
27 S13044   520764 1729    
28 S13051   107452  354    
29 S13052   100314  250    
30 S14006   712839 2284    
31 S14010   790361 2290    
32 S14011   805724 2434    
33 S14012   720600 2412    
34 S14021   699511 2231    
35 S14022   651328 2047    
36 S14023   727630 2258    
37 S14024   736323 2417    
38 S14025   755581 2394    
39 S14026   824537 2503    
40 S14027   714550 2303    
41 S14028   786145 2458    
42 S14033   725318 2282    
43 S14034   653681 2024    
44 S14036   203694  635    
45 S14040   701858 2233    
46 S14042   534689 1935    
47 S14045   458484 1398    
48 S14046   412646 1354    
49 S14047   338329 1041    
50 S14048   234276  769    
51 S14049   230808  788    
52 S14050   167090  580

---
> R-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [25]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(max_ymd = max(sales_ymd), .groups = "drop") %>%
    slice(1:10)

customer_id    max_ymd 
1  CS001113000004 20190308
2  CS001114000005 20190731
3  CS001115000010 20190405
4  CS001205000004 20190625
5  CS001205000006 20190224
6  CS001211000025 20190322
7  CS001212000027 20170127
8  CS001212000031 20180906
9  CS001212000046 20170811
10 CS001212000070 20191018

---
> R-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [26]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(min_ymd = min(sales_ymd), .groups = "drop") %>%
    slice(1:10)

customer_id    min_ymd 
1  CS001113000004 20190308
2  CS001114000005 20180503
3  CS001115000010 20171228
4  CS001205000004 20170914
5  CS001205000006 20180207
6  CS001211000025 20190322
7  CS001212000027 20170127
8  CS001212000031 20180906
9  CS001212000046 20170811
10 CS001212000070 20191018

---
> R-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [27]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(max_ymd = max(sales_ymd)
             ,min_ymd = min(sales_ymd), .groups = "drop") %>%
    filter(max_ymd != min_ymd) %>%
    slice(1:10)

customer_id    max_ymd  min_ymd 
1  CS001114000005 20190731 20180503
2  CS001115000010 20190405 20171228
3  CS001205000004 20190625 20170914
4  CS001205000006 20190224 20180207
5  CS001214000009 20190902 20170306
6  CS001214000017 20191006 20180828
7  CS001214000048 20190929 20171109
8  CS001214000052 20190617 20180208
9  CS001215000005 20181021 20170206
10 CS001215000040 20171022 20170214

---
> R-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [28]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(mean_amount = mean(amount), .groups = "drop") %>%
    arrange(desc(mean_amount)) %>%
    slice(1:5)

store_cd mean_amount
1 S13052   402.8675   
2 S13015   351.1120   
3 S13003   350.9155   
4 S14010   348.7913   
5 S13001   348.4704

---
> R-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [29]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(median_amount = median(amount), .groups = "drop") %>%
    arrange(desc(median_amount)) %>%
    slice(1:5)

store_cd median_amount
1 S13052   190          
2 S14010   188          
3 S14050   185          
4 S13003   180          
5 S13018   180

---
> R-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [30]:
# コード例1
df_receipt %>%
  group_by(store_cd, product_cd) %>%
  summarise(cnt = n(), .groups = "drop_last") %>%
  filter(cnt == cnt %>% max()) %>%
  ungroup() %>%
  slice(1:10)

store_cd product_cd cnt
1  S12007   P060303001  72
2  S12013   P060303001 107
3  S12014   P060303001  65
4  S12029   P060303001  92
5  S12030   P060303001 115
6  S13001   P060303001  67
7  S13002   P060303001  78
8  S13003   P071401001  65
9  S13004   P060303001  88
10 S13005   P040503001  36

In [31]:
# コード例2： which.max()を使用（最頻値複数発生時、どれか一つに絞られる）
# 件数の表示は省略
table_product <-table(df_receipt$store_cd, df_receipt$product_cd)

store <- names(table_product[,1])

mode_product <- c()

for (i in 1:length(store)){
    mode_product[i] <- names(which.max(table_product[i,]))
}

data.frame(store_cd = store, product_cd = mode_product) %>%
  slice(1:10)

store_cd product_cd
1  S12007   P060303001
2  S12013   P060303001
3  S12014   P060303001
4  S12029   P060303001
5  S12030   P060303001
6  S13001   P060303001
7  S13002   P060303001
8  S13003   P071401001
9  S13004   P060303001
10 S13005   P040503001

---
> R-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [32]:
var_sample <- function(x){ var(x) * (length(x) - 1) / length(x) }

df_receipt %>%
    group_by(store_cd) %>%
    summarise(var_amount = var_sample(amount), .groups = "drop") %>%
    arrange(desc(var_amount)) %>%
    slice(1:5)

store_cd var_amount
1 S13052   440088.7  
2 S14011   306314.6  
3 S14034   296920.1  
4 S13001   295432.0  
5 S13015   295294.4

---
> R-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [33]:
var_sample <- function(x){ var(x)*(length(x)-1)/length(x) }

std_sample <- function(x){ sqrt(var_sample(x)) }

head(
    df_receipt %>%
        group_by(store_cd) %>%
        summarise(std_amount = std_sample(amount), .groups = "drop") %>%
        arrange(desc(std_amount))
    , n = 5
)

store_cd std_amount
1 S13052   663.3918  
2 S14011   553.4569  
3 S14034   544.9037  
4 S13001   543.5366  
5 S13015   543.4099

---
> R-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [34]:
df_receipt %>%
    summarise(amount_25per = quantile(amount, 0.25),
              amount_50per = quantile(amount, 0.5),
              amount_75per = quantile(amount, 0.75),
              amount_100per = quantile(amount, 1.0))

amount_25per amount_50per amount_75per amount_100per
1 102          170          288          10925

---
> R-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [35]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(mean_amount = mean(amount), .groups = "drop") %>%
    filter(mean_amount >= 330)

store_cd mean_amount
1  S12013   330.1941   
2  S13001   348.4704   
3  S13003   350.9155   
4  S13004   330.9439   
5  S13015   351.1120   
6  S13019   330.2086   
7  S13020   337.8799   
8  S13052   402.8675   
9  S14010   348.7913   
10 S14011   335.7183   
11 S14026   332.3406   
12 S14045   330.0821   
13 S14047   330.0771

---
> R-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [36]:
df_mean <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    summarise(mean_amount = mean(sum_amount))

df_mean$mean_amount

[1] 2547.742

---
> R-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [37]:
df_sum <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

df_mean <- df_sum %>%
    summarise(mean_amount = mean(sum_amount))

df_sum %>%
    filter(sum_amount >= df_mean$mean_amount) %>%
    slice(1:10)

customer_id    sum_amount
1  CS001115000010 3044      
2  CS001205000006 3337      
3  CS001214000009 4685      
4  CS001214000017 4132      
5  CS001214000052 5639      
6  CS001215000040 3496      
7  CS001304000006 3726      
8  CS001305000005 3485      
9  CS001305000011 4370      
10 CS001315000180 3300

---
> R-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [38]:
inner_join(df_receipt, 
           df_store[c("store_cd", "store_name")], by = "store_cd") %>%
    slice(1:10)

sales_ymd sales_epoch store_cd receipt_no receipt_sub_no customer_id   
1  20181103  1541203200  S14006    112       1              CS006214000001
2  20181118  1542499200  S13008   1132       2              CS008415000097
3  20170712  1499817600  S14028   1102       1              CS028414000014
4  20190205  1549324800  S14042   1132       1              ZZ000000000000
5  20180821  1534809600  S14025   1102       2              CS025415000050
6  20190605  1559692800  S13003   1112       1              CS003515000195
7  20181205  1543968000  S14024   1102       2              CS024514000042
8  20190922  1569110400  S14040   1102       1              CS040415000178
9  20170504  1493856000  S13020   1112       2              ZZ000000000000
10 20191010  1570665600  S14027   1102       1              CS027514000015
   product_cd quantity amount store_name
1  P070305012 1        158    葛が谷店  
2  P070701017 1         81    成城店    
3  P060101005 1        170    二ツ橋店  
4  P050301001 1         25    新山下店  
5  P060102007 1         90    大和店    
6  P050102002 1        138    狛江店    
7  P080101005 1         30    三田店    
8  P070501004 1        128    長津田店  
9  P071302010 1        770    十条仲原店
10 P071101003 1        680    南藤沢店

---
> R-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [39]:
inner_join(df_product,
           df_category[c("category_small_cd", "category_small_name")],
           by = "category_small_cd") %>%
    slice(1:10)

product_cd category_major_cd category_medium_cd category_small_cd unit_price
1  P040101001 04                0401               040101            198       
2  P040101002 04                0401               040101            218       
3  P040101003 04                0401               040101            230       
4  P040101004 04                0401               040101            248       
5  P040101005 04                0401               040101            268       
6  P040101006 04                0401               040101            298       
7  P040101007 04                0401               040101            338       
8  P040101008 04                0401               040101            420       
9  P040101009 04                0401               040101            498       
10 P040101010 04                0401               040101            580       
   unit_cost category_small_name
1  149       弁当類             
2  164       弁当類             
3  173       弁当類             
4  186       弁当類             
5  201       弁当類             
6  224       弁当類             
7  254       弁当類             
8  315       弁当類             
9  374       弁当類             
10 435       弁当類

---
> R-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [40]:
df_sum <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

df_target <- df_customer %>%
    filter(gender_cd == "1" & !grepl("^Z", customer_id))

left_join(df_target["customer_id"], df_sum, by = "customer_id") %>%
    replace_na(list(sum_amount = 0)) %>%
    slice(1:10)

customer_id    sum_amount
1  CS021313000114    0      
2  CS031415000172 5088      
3  CS028811000001    0      
4  CS001215000145  875      
5  CS015414000103 3122      
6  CS033513000180  868      
7  CS035614000014    0      
8  CS011215000048 3444      
9  CS009413000079    0      
10 CS040412000191  210

---
> R-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [41]:
# コード例１
df_data <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id)

df_cnt <- df_data %>%
    summarise(come_days = n_distinct(sales_ymd), .groups = "drop") %>%
    arrange(desc(come_days), customer_id) %>%
    slice(1:20)

df_sum <- df_data %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    arrange(desc(sum_amount)) %>%
    slice(1:20)

full_join(df_cnt, df_sum, by = "customer_id")

customer_id    come_days sum_amount
1  CS040214000008 23           NA     
2  CS010214000010 22        18585     
3  CS015415000185 22        20153     
4  CS010214000002 21           NA     
5  CS028415000007 21        19127     
6  CS016415000141 20        18372     
7  CS017415000097 20        23086     
8  CS014214000023 19           NA     
9  CS021514000045 19           NA     
10 CS021515000172 19           NA     
11 CS022515000226 19           NA     
12 CS031414000051 19        19202     
13 CS039414000052 19           NA     
14 CS007515000107 18           NA     
15 CS014415000077 18           NA     
16 CS021515000056 18           NA     
17 CS021515000211 18           NA     
18 CS022515000028 18           NA     
19 CS030214000008 18           NA     
20 CS031414000073 18           NA     
21 CS001605000009 NA        18925     
22 CS006515000023 NA        18372     
23 CS011414000106 NA        18338     
24 CS038415000104 NA        17847     
25 CS035414000024 NA        17615     
26 CS021515000089 NA        17580     
27 CS032414000072 NA        16563     
28 CS016415000101 NA        16348     
29 CS011415000006 NA        16094     
30 CS034415000047 NA        16083     
31 CS007514000094 NA        15735     
32 CS009414000059 NA        15492     
33 CS030415000034 NA        15468     
34 CS015515000034 NA        15300

In [42]:
# コード例2
# slice_maxを使用したコード例
# with_ties = FALSEをwith_ties = TRUEに書き換えると同一順位を含める形に変更できる

df_data <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id)

df_cnt <- df_data %>%
    summarise(come_days = n_distinct(sales_ymd)) %>%
    slice_max(come_days, n = 20, with_ties = FALSE)

df_sum <- df_data %>%
    summarise(sum_amount = sum(amount)) %>%
    slice_max(sum_amount, n = 20, with_ties = FALSE)

full_join(df_cnt, df_sum, by = "customer_id")

customer_id    come_days sum_amount
1  CS040214000008 23           NA     
2  CS010214000010 22        18585     
3  CS015415000185 22        20153     
4  CS010214000002 21           NA     
5  CS028415000007 21        19127     
6  CS016415000141 20        18372     
7  CS017415000097 20        23086     
8  CS014214000023 19           NA     
9  CS021514000045 19           NA     
10 CS021515000172 19           NA     
11 CS022515000226 19           NA     
12 CS031414000051 19        19202     
13 CS039414000052 19           NA     
14 CS007515000107 18           NA     
15 CS014415000077 18           NA     
16 CS021515000056 18           NA     
17 CS021515000211 18           NA     
18 CS022515000028 18           NA     
19 CS030214000008 18           NA     
20 CS031414000073 18           NA     
21 CS001605000009 NA        18925     
22 CS006515000023 NA        18372     
23 CS011414000106 NA        18338     
24 CS038415000104 NA        17847     
25 CS035414000024 NA        17615     
26 CS021515000089 NA        17580     
27 CS032414000072 NA        16563     
28 CS016415000101 NA        16348     
29 CS011415000006 NA        16094     
30 CS034415000047 NA        16083     
31 CS007514000094 NA        15735     
32 CS009414000059 NA        15492     
33 CS030415000034 NA        15468     
34 CS015515000034 NA        15300

---
> R-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [43]:
df_store_tmp <- df_store

df_product_tmp <- df_product

df_store_tmp["key"] <- 0

df_product_tmp["key"] <- 0

nrow(full_join(df_store_tmp, df_product_tmp, by = "key", relationship = "many-to-many"))

[1] 531590

---
> R-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [44]:
df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(lag_ymd = lag(sales_ymd), 
           lag_amount = lag(sum_amount), 
           diff_amount = sum_amount - lag_amount) %>%
    slice(1:10)

sales_ymd sum_amount lag_ymd  lag_amount diff_amount
1  20170101  33723            NA    NA         NA      
2  20170102  24165      20170101 33723      -9558      
3  20170103  27503      20170102 24165       3338      
4  20170104  36165      20170103 27503       8662      
5  20170105  37830      20170104 36165       1665      
6  20170106  32387      20170105 37830      -5443      
7  20170107  23415      20170106 32387      -8972      
8  20170108  24737      20170107 23415       1322      
9  20170109  26718      20170108 24737       1981      
10 20170110  20143      20170109 26718      -6575

---
> R-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [45]:
# コード例1:縦持ちケース
df_sum <-df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

for (i in 1:3) {
    df_tmp <- df_sum %>%
        mutate(lag_ymd = lag(sales_ymd, n = i), 
               lag_amount = lag(sum_amount, n = i))
    if (i == 1) {
        df_lag <- df_tmp
    }
    else {
        df_lag <- rbind(df_lag, df_tmp)
    }
}

df_lag %>%
    drop_na(everything()) %>%
    arrange(sales_ymd, lag_ymd) %>%
    slice(1:10)

sales_ymd sum_amount lag_ymd  lag_amount
1  20170102  24165      20170101 33723     
2  20170103  27503      20170101 33723     
3  20170103  27503      20170102 24165     
4  20170104  36165      20170101 33723     
5  20170104  36165      20170102 24165     
6  20170104  36165      20170103 27503     
7  20170105  37830      20170102 24165     
8  20170105  37830      20170103 27503     
9  20170105  37830      20170104 36165     
10 20170106  32387      20170103 27503

In [46]:
# コード例2:横持ちケース
df_sum <-df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

for (i in 1:3) {
    col_name_1 <- paste("lag_ymd_", i , sep="_")
    col_name_2 <- paste("lag_amount", i , sep="_")
    df_tmp <- df_sum %>%
        mutate(!!col_name_1 := lag(sales_ymd, n　=　i), 
               !!col_name_2 := lag(sum_amount, n　=　i))
    if (i == 1)　{
        df_lag <- df_tmp
    }
    else {
        df_lag <- cbind(df_lag, df_tmp[c(col_name_1, col_name_2)])
    }
}

df_lag %>%
    drop_na(everything()) %>%
    arrange(sales_ymd) %>%
    slice(1:10)

sales_ymd sum_amount lag_ymd__1 lag_amount_1 lag_ymd__2 lag_amount_2
1  20170104  36165      20170103   27503        20170102   24165       
2  20170105  37830      20170104   36165        20170103   27503       
3  20170106  32387      20170105   37830        20170104   36165       
4  20170107  23415      20170106   32387        20170105   37830       
5  20170108  24737      20170107   23415        20170106   32387       
6  20170109  26718      20170108   24737        20170107   23415       
7  20170110  20143      20170109   26718        20170108   24737       
8  20170111  24287      20170110   20143        20170109   26718       
9  20170112  23526      20170111   24287        20170110   20143       
10 20170113  28004      20170112   23526        20170111   24287       
   lag_ymd__3 lag_amount_3
1  20170101   33723       
2  20170102   24165       
3  20170103   27503       
4  20170104   36165       
5  20170105   37830       
6  20170106   32387       
7  20170107   23415       
8  20170108   24737       
9  20170109   26718       
10 20170110   20143

---
> R-043: レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [47]:
df_sales_summary <- df_customer[c("customer_id", "gender_cd", "age")] %>%
    mutate(era = trunc(age / 10) * 10) %>%
    inner_join(df_receipt, by = "customer_id") %>%
    group_by(gender_cd, era) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    spread(
        gender_cd, 
        sum_amount, 
        fill = 0) %>%
    rename(male = "0", female = "1", unknown = "9")

df_sales_summary

era male   female  unknown
1 10    1591  149836   4317 
2 20   72940 1363724  44328 
3 30  177322  693047  50441 
4 40   19355 9320791 483512 
5 50   54320 6685192 342923 
6 60  272469  987741  71418 
7 70   13435   29764   2427 
8 80   46360  262923   5111 
9 90       0    6260      0

---
> R-044: 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [48]:
df_sales_summary　%>%
    gather(key = gender_cd, 
        value = sum_amount, male, female, unknown) %>%
    mutate(gender_cd = case_when(
        gender_cd == "male"  ~ "00",
        gender_cd == "female"  ~ "01",
        gender_cd == "unknown"  ~ "99",
    ))

era gender_cd sum_amount
1  10  00           1591   
2  20  00          72940   
3  30  00         177322   
4  40  00          19355   
5  50  00          54320   
6  60  00         272469   
7  70  00          13435   
8  80  00          46360   
9  90  00              0   
10 10  01         149836   
11 20  01        1363724   
12 30  01         693047   
13 40  01        9320791   
14 50  01        6685192   
15 60  01         987741   
16 70  01          29764   
17 80  01         262923   
18 90  01           6260   
19 10  99           4317   
20 20  99          44328   
21 30  99          50441   
22 40  99         483512   
23 50  99         342923   
24 60  99          71418   
25 70  99           2427   
26 80  99           5111   
27 90  99              0

---
> R-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [49]:
df_tmp <- cbind(df_customer["customer_id"], 
                strftime(
                    df_customer$birth_day,
                    format = "%Y%m%d"
                ))

colnames(df_tmp) <- c("customer_id", "birth_day")

head(df_tmp,10)

customer_id    birth_day
1  CS021313000114 19810429 
2  CS037613000071 19520401 
3  CS031415000172 19761004 
4  CS028811000001 19330327 
5  CS001215000145 19950329 
6  CS020401000016 19740915 
7  CS015414000103 19770809 
8  CS029403000008 19730817 
9  CS015804000004 19310502 
10 CS033513000180 19620711

---
> R-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [50]:
df_tmp <- cbind(df_customer["customer_id"], 
                strptime(df_customer$application_date, "%Y%m%d"))

colnames(df_tmp) <- c("customer_id", "application_date")

head(df_tmp, 10)

customer_id    application_date
1  CS021313000114 2015-09-05      
2  CS037613000071 2015-04-14      
3  CS031415000172 2015-05-29      
4  CS028811000001 2016-01-15      
5  CS001215000145 2017-06-05      
6  CS020401000016 2015-02-25      
7  CS015414000103 2015-07-22      
8  CS029403000008 2015-05-15      
9  CS015804000004 2015-06-07      
10 CS033513000180 2015-07-28

---
> R-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [51]:
df_tmp <- cbind(df_receipt[c("receipt_no", "receipt_sub_no")], 
                strptime(as.character(df_receipt$sales_ymd), "%Y%m%d"))

colnames(df_tmp) <- c("receipt_no", "receipt_sub_no", "sales_ymd")

head(df_tmp, 10)

receipt_no receipt_sub_no sales_ymd 
1   112       1              2018-11-03
2  1132       2              2018-11-18
3  1102       1              2017-07-12
4  1132       1              2019-02-05
5  1102       2              2018-08-21
6  1112       1              2019-06-05
7  1102       2              2018-12-05
8  1102       1              2019-09-22
9  1112       2              2017-05-04
10 1102       1              2019-10-10

---
> R-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [52]:
df_tmp <- cbind(df_receipt[c("receipt_no", "receipt_sub_no")], 
                as.POSIXct(df_receipt$sales_epoch, origin="1970-01-01"))

colnames(df_tmp) <- c("receipt_no", "receipt_sub_no", "sales_ymd")

head(df_tmp, 10)

receipt_no receipt_sub_no sales_ymd 
1   112       1              2018-11-03
2  1132       2              2018-11-18
3  1102       1              2017-07-12
4  1132       1              2019-02-05
5  1102       2              2018-08-21
6  1112       1              2019-06-05
7  1102       2              2018-12-05
8  1102       1              2019-09-22
9  1112       2              2017-05-04
10 1102       1              2019-10-10

---
> R-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [53]:
df_tmp <- cbind(df_receipt[c("receipt_no", "receipt_sub_no")], 
                substring(as.POSIXct(df_receipt$sales_epoch, 
                                     origin = "1970-01-01"), 1, 4))

colnames(df_tmp) <- c("receipt_no", "receipt_sub_no", "sales_year")

head(df_tmp, 10)

receipt_no receipt_sub_no sales_year
1   112       1              2018      
2  1132       2              2018      
3  1102       1              2017      
4  1132       1              2019      
5  1102       2              2018      
6  1112       1              2019      
7  1102       2              2018      
8  1102       1              2019      
9  1112       2              2017      
10 1102       1              2019

---
> R-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [54]:
df_tmp <- cbind(df_receipt[c("receipt_no", "receipt_sub_no")], 
                substring(as.POSIXct(df_receipt$sales_epoch, 
                                     origin = "1970-01-01"), 6, 7))

colnames(df_tmp) <- c("receipt_no", "receipt_sub_no", "sales_ymd")

head(df_tmp, 10)

receipt_no receipt_sub_no sales_ymd
1   112       1              11       
2  1132       2              11       
3  1102       1              07       
4  1132       1              02       
5  1102       2              08       
6  1112       1              06       
7  1102       2              12       
8  1102       1              09       
9  1112       2              05       
10 1102       1              10

---
> R-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [55]:
df_tmp <- cbind(df_receipt[c("receipt_no", "receipt_sub_no")], 
                substring(as.POSIXct(df_receipt$sales_epoch, 
                                     origin = "1970-01-01"), 9, 10))

colnames(df_tmp) <- c("receipt_no", "receipt_sub_no", "sales_day")

head(df_tmp, 10)

receipt_no receipt_sub_no sales_day
1   112       1              03       
2  1132       2              18       
3  1102       1              12       
4  1132       1              05       
5  1102       2              21       
6  1112       1              05       
7  1102       2              05       
8  1102       1              22       
9  1112       2              04       
10 1102       1              10

---
> R-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [56]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount=sum(amount), .groups = "drop") %>%
    mutate(sales_flg = ifelse(sum_amount > 2000, 1, 0))　%>%
    slice(1:10)

customer_id    sum_amount sales_flg
1  CS001113000004 1298       0        
2  CS001114000005  626       0        
3  CS001115000010 3044       1        
4  CS001205000004 1988       0        
5  CS001205000006 3337       1        
6  CS001211000025  456       0        
7  CS001212000027  448       0        
8  CS001212000031  296       0        
9  CS001212000046  228       0        
10 CS001212000070  456       0

---
> R-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [57]:
df_customer[c("customer_id", "postal_cd")] %>%
    mutate(postal_flg = 
           ifelse(100 <= as.integer(str_sub(postal_cd, start = 1, end = 3)) 
                  & as.integer(str_sub(postal_cd, start = 1, end = 3)) <= 209, 
                  1, 0)) %>%
    inner_join(df_receipt, by = "customer_id") %>%
    group_by(postal_flg) %>%
    summarise(customer_cnt = n_distinct(customer_id), .groups = "drop")

postal_flg customer_cnt
1 0          3906        
2 1          4400

---
> R-054:  顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [58]:
# コード例1（固定で切り出す）
df_customer %>%
    mutate(prefecture_cd = 
           case_when(
               str_sub(address, start = 1, end = 3) == "埼玉県" ~ "11",
               str_sub(address, start = 1, end = 3) == "千葉県" ~ "12",
               str_sub(address, start = 1, end = 3) == "東京都" ~ "13",
               str_sub(address, start = 1, end = 3) == "神奈川" ~ "14")) %>%
    select(customer_id, address, prefecture_cd) %>%
    slice(1:10)

customer_id    address                            prefecture_cd
1  CS021313000114 神奈川県伊勢原市粟窪**********     14           
2  CS037613000071 東京都江東区南砂**********         13           
3  CS031415000172 東京都渋谷区代々木**********       13           
4  CS028811000001 神奈川県横浜市泉区和泉町********** 14           
5  CS001215000145 東京都大田区仲六郷**********       13           
6  CS020401000016 東京都板橋区若木**********         13           
7  CS015414000103 東京都江東区北砂**********         13           
8  CS029403000008 千葉県浦安市海楽**********         12           
9  CS015804000004 東京都江東区北砂**********         13           
10 CS033513000180 神奈川県横浜市旭区善部町********** 14

In [59]:
# コード例2（正規表現を使う）
df_customer %>%
  mutate(prefecture_cd = 
    case_when(
      str_extract(address, pattern = "^.*?[都道府県]") == "埼玉県" ~ "11",
      str_extract(address, pattern = "^.*?[都道府県]") == "千葉県" ~ "12",
      str_extract(address, pattern = "^.*?[都道府県]") == "東京都" ~ "13",
      str_extract(address, pattern = "^.*?[都道府県]") == "神奈川県" ~ "14")) %>%
    select(customer_id, address, prefecture_cd) %>%
    slice(1:10)

customer_id    address                            prefecture_cd
1  CS021313000114 神奈川県伊勢原市粟窪**********     14           
2  CS037613000071 東京都江東区南砂**********         13           
3  CS031415000172 東京都渋谷区代々木**********       13           
4  CS028811000001 神奈川県横浜市泉区和泉町********** 14           
5  CS001215000145 東京都大田区仲六郷**********       13           
6  CS020401000016 東京都板橋区若木**********         13           
7  CS015414000103 東京都江東区北砂**********         13           
8  CS029403000008 千葉県浦安市海楽**********         12           
9  CS015804000004 東京都江東区北砂**********         13           
10 CS033513000180 神奈川県横浜市旭区善部町********** 14

---
> R-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [60]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(pct_group = case_when(
        sum_amount < quantile(sum_amount)[2]  ~ "1",
        sum_amount < quantile(sum_amount)[3]  ~ "2",
        sum_amount < quantile(sum_amount)[4]  ~ "3",
        quantile(sum_amount)[4] <= sum_amount ~ "4"
    )) %>%
    slice(1:10)

customer_id    sum_amount pct_group
1  CS001113000004 1298       2        
2  CS001114000005  626       2        
3  CS001115000010 3044       3        
4  CS001205000004 1988       3        
5  CS001205000006 3337       3        
6  CS001211000025  456       1        
7  CS001212000027  448       1        
8  CS001212000031  296       1        
9  CS001212000046  228       1        
10 CS001212000070  456       1

---
> R-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [61]:
# コード例1

df_customer[c("customer_id", "birth_day", "age")] %>%
    mutate(era = trunc(age / 10) * 10) %>%
    mutate(era = case_when(
        era < 60 ~ era,
        era >= 60 ~　60
    )) %>%
    slice(1:10)

customer_id    birth_day  age era
1  CS021313000114 1981-04-29 37  30 
2  CS037613000071 1952-04-01 66  60 
3  CS031415000172 1976-10-04 42  40 
4  CS028811000001 1933-03-27 86  60 
5  CS001215000145 1995-03-29 24  20 
6  CS020401000016 1974-09-15 44  40 
7  CS015414000103 1977-08-09 41  40 
8  CS029403000008 1973-08-17 45  40 
9  CS015804000004 1931-05-02 87  60 
10 CS033513000180 1962-07-11 56  50

In [62]:
# コード例2（minを使う場合）

df_customer[c("customer_id", "birth_day","age")] %>%
    mutate(era = sapply(1:nrow(df_customer), 
                function(x){
                    return (min(trunc(.[x, 3] / 10) * 10, 60))
                })) %>%
    slice(1:10)

customer_id    birth_day  age era
1  CS021313000114 1981-04-29 37  30 
2  CS037613000071 1952-04-01 66  60 
3  CS031415000172 1976-10-04 42  40 
4  CS028811000001 1933-03-27 86  60 
5  CS001215000145 1995-03-29 24  20 
6  CS020401000016 1974-09-15 44  40 
7  CS015414000103 1977-08-09 41  40 
8  CS029403000008 1973-08-17 45  40 
9  CS015804000004 1931-05-02 87  60 
10 CS033513000180 1962-07-11 56  50

---
> R-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [63]:
# 性別コード１桁と年代コード２桁を連結した性年代コードを生成する

df_customer[c("customer_id", "gender_cd", "birth_day", "age")] %>%
    mutate(era = trunc(age / 10) * 10) %>%
    mutate(era = case_when(
        era < 60 ~ formatC(era, width = 2,flag = "0"),
        era >= 60 ~　formatC(60, width = 2,flag = "0"))) %>%
    mutate(gender_era = paste(gender_cd, era, sep = "")) %>%
    slice(1:10)

customer_id    gender_cd birth_day  age era gender_era
1  CS021313000114 1         1981-04-29 37  30  130       
2  CS037613000071 9         1952-04-01 66  60  960       
3  CS031415000172 1         1976-10-04 42  40  140       
4  CS028811000001 1         1933-03-27 86  60  160       
5  CS001215000145 1         1995-03-29 24  20  120       
6  CS020401000016 0         1974-09-15 44  40  040       
7  CS015414000103 1         1977-08-09 41  40  140       
8  CS029403000008 0         1973-08-17 45  40  040       
9  CS015804000004 0         1931-05-02 87  60  060       
10 CS033513000180 1         1962-07-11 56  50  150

---
> R-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [64]:
# コード例1（すべてのコード値を項目化）
dummy_gender_model <- dummyVars(~gender_cd, 
                                data = df_customer, fullRank = FALSE)

dummy_gender <- predict(dummy_gender_model, df_customer)

head(cbind(df_customer["customer_id"], dummy_gender), 10)

customer_id    gender_cd0 gender_cd1 gender_cd9
1  CS021313000114 0          1          0         
2  CS037613000071 0          0          1         
3  CS031415000172 0          1          0         
4  CS028811000001 0          1          0         
5  CS001215000145 0          1          0         
6  CS020401000016 1          0          0         
7  CS015414000103 0          1          0         
8  CS029403000008 1          0          0         
9  CS015804000004 1          0          0         
10 CS033513000180 0          1          0

In [65]:
# コード例2（fullRank=TRUEとすることで項目を一つ削ることができる）
dummy_gender_model <- dummyVars(~gender_cd, 
                                data = df_customer, fullRank = TRUE)

dummy_gender <- predict(dummy_gender_model, df_customer)

head(cbind(df_customer["customer_id"], dummy_gender), 3)

customer_id    gender_cd1 gender_cd9
1 CS021313000114 1          0         
2 CS037613000071 0          1         
3 CS031415000172 1          0

---
> R-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [66]:
# Rのscaleを使用しているため不偏分散の平方根による標準偏差で標準化される
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(std_amount = scale(sum_amount, 
                              center = TRUE, scale = TRUE)) %>%
    slice(1:10)

customer_id    sum_amount std_amount
1  CS001113000004 1298       -0.4593502
2  CS001114000005  626       -0.7063478
3  CS001115000010 3044        0.1824025
4  CS001205000004 1988       -0.2057366
5  CS001205000006 3337        0.2900964
6  CS001211000025  456       -0.7688324
7  CS001212000027  448       -0.7717728
8  CS001212000031  296       -0.8276413
9  CS001212000046  228       -0.8526351
10 CS001212000070  456       -0.7688324

---
> R-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [67]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(
        scale_amount = scale(sum_amount, 
                             center = min(sum_amount), 
                             scale = max(sum_amount) - min(sum_amount)))%>%
    slice(1:10)

customer_id    sum_amount scale_amount
1  CS001113000004 1298       0.053354188 
2  CS001114000005  626       0.024157108 
3  CS001115000010 3044       0.129214460 
4  CS001205000004 1988       0.083333333 
5  CS001205000006 3337       0.141944734 
6  CS001211000025  456       0.016770942 
7  CS001212000027  448       0.016423358 
8  CS001212000031  296       0.009819256 
9  CS001212000046  228       0.006864790 
10 CS001212000070  456       0.016770942

---
> R-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [68]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(log_amount = log((sum_amount + 0.5), base = 10)) %>%
    slice(1:10)

customer_id    sum_amount log_amount
1  CS001113000004 1298       3.113442  
2  CS001114000005  626       2.796921  
3  CS001115000010 3044       3.483516  
4  CS001205000004 1988       3.298526  
5  CS001205000006 3337       3.523421  
6  CS001211000025  456       2.659441  
7  CS001212000027  448       2.651762  
8  CS001212000031  296       2.472025  
9  CS001212000046  228       2.358886  
10 CS001212000070  456       2.659441

---
> R-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [69]:
df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(log_amount = log(sum_amount + 0.5)) %>%
    slice(1:10)

customer_id    sum_amount log_amount
1  CS001113000004 1298       7.168965  
2  CS001114000005  626       6.440149  
3  CS001115000010 3044       8.021092  
4  CS001205000004 1988       7.595136  
5  CS001205000006 3337       8.112977  
6  CS001211000025  456       6.123589  
7  CS001212000027  448       6.105909  
8  CS001212000031  296       5.692047  
9  CS001212000046  228       5.431536  
10 CS001212000070  456       6.123589

---
> R-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [70]:
df_product %>%
    mutate(unit_profit = unit_price - unit_cost) %>%
    slice(1:10)

product_cd category_major_cd category_medium_cd category_small_cd unit_price
1  P040101001 04                0401               040101            198       
2  P040101002 04                0401               040101            218       
3  P040101003 04                0401               040101            230       
4  P040101004 04                0401               040101            248       
5  P040101005 04                0401               040101            268       
6  P040101006 04                0401               040101            298       
7  P040101007 04                0401               040101            338       
8  P040101008 04                0401               040101            420       
9  P040101009 04                0401               040101            498       
10 P040101010 04                0401               040101            580       
   unit_cost unit_profit
1  149        49        
2  164        54        
3  173        57        
4  186        62        
5  201        67        
6  224        74        
7  254        84        
8  315       105        
9  374       124        
10 435       145

---
> R-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [71]:
df_product %>%
    mutate(unit_profit_rate = (unit_price - unit_cost) / unit_price) %>%
    summarise(total_mean = mean(unit_profit_rate, na.rm = TRUE))

total_mean
1 0.2491139

---
> R-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [72]:
df_product[c("product_cd", "unit_price", "unit_cost")] %>%
    mutate(new_price = trunc(unit_cost / 0.7)) %>%
    mutate(new_profit_rate = (new_price - unit_cost)/ new_price) %>%
    slice(1:10)

product_cd unit_price unit_cost new_price new_profit_rate
1  P040101001 198        149       212       0.2971698      
2  P040101002 218        164       234       0.2991453      
3  P040101003 230        173       247       0.2995951      
4  P040101004 248        186       265       0.2981132      
5  P040101005 268        201       287       0.2996516      
6  P040101006 298        224       320       0.3000000      
7  P040101007 338        254       362       0.2983425      
8  P040101008 420        315       450       0.3000000      
9  P040101009 498        374       534       0.2996255      
10 P040101010 580        435       621       0.2995169

---
> R-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [73]:
df_product[c("product_cd", "unit_price", "unit_cost")] %>%
    mutate(new_price = round(unit_cost / 0.7)) %>%
    mutate(new_profit_rate = (new_price - unit_cost) / new_price) %>%
    slice(1:10)

product_cd unit_price unit_cost new_price new_profit_rate
1  P040101001 198        149       213       0.3004695      
2  P040101002 218        164       234       0.2991453      
3  P040101003 230        173       247       0.2995951      
4  P040101004 248        186       266       0.3007519      
5  P040101005 268        201       287       0.2996516      
6  P040101006 298        224       320       0.3000000      
7  P040101007 338        254       363       0.3002755      
8  P040101008 420        315       450       0.3000000      
9  P040101009 498        374       534       0.2996255      
10 P040101010 580        435       621       0.2995169

---
> R-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [74]:
df_product[c("product_cd", "unit_price", "unit_cost")] %>%
    mutate(new_price = ceiling(unit_cost / 0.7)) %>%
    mutate(new_profit_rate = (new_price - unit_cost) / new_price) %>%
    slice(1:10)

product_cd unit_price unit_cost new_price new_profit_rate
1  P040101001 198        149       213       0.3004695      
2  P040101002 218        164       235       0.3021277      
3  P040101003 230        173       248       0.3024194      
4  P040101004 248        186       266       0.3007519      
5  P040101005 268        201       288       0.3020833      
6  P040101006 298        224       320       0.3000000      
7  P040101007 338        254       363       0.3002755      
8  P040101008 420        315       451       0.3015521      
9  P040101009 498        374       535       0.3009346      
10 P040101010 580        435       622       0.3006431

---
> R-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [75]:
head(cbind(product_cd = df_product$product_cd,
           unit_price = df_product$unit_price,
           tax_price = trunc(df_product$unit_price * 1.1)), 10)

product_cd unit_price tax_price
 [1,] P040101001 198        217      
 [2,] P040101002 218        239      
 [3,] P040101003 230        253      
 [4,] P040101004 248        272      
 [5,] P040101005 268        294      
 [6,] P040101006 298        327      
 [7,] P040101007 338        371      
 [8,] P040101008 420        462      
 [9,] P040101009 498        547      
[10,] P040101010 580        638

---
> R-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [76]:
# コード例1
df_tmp_1 <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_all=sum(amount))

df_tmp_2 <- inner_join(df_receipt, 
                       df_product[c("product_cd","category_major_cd")], 
                       by="product_cd") %>% 
    filter(category_major_cd == "07") %>%
    group_by(customer_id) %>%
    summarise(sum_07 = sum(amount), .groups = "drop")

inner_join(df_tmp_1, df_tmp_2, by = "customer_id") %>%
    mutate(sales_rate = sum_07 / sum_all) %>%
    slice(1:10)

customer_id    sum_all sum_07 sales_rate
1  CS001113000004 1298    1298   1.0000000 
2  CS001114000005  626     486   0.7763578 
3  CS001115000010 3044    2694   0.8850197 
4  CS001205000004 1988     346   0.1740443 
5  CS001205000006 3337    2004   0.6005394 
6  CS001212000027  448     200   0.4464286 
7  CS001212000031  296     296   1.0000000 
8  CS001212000046  228     108   0.4736842 
9  CS001212000070  456     308   0.6754386 
10 CS001213000018  243     145   0.5967078

In [77]:
# コード例2（%>%ですべての処理をつなげる）

inner_join(df_receipt, df_product[c("product_cd","category_major_cd")], 
           by="product_cd") %>%
    filter(category_major_cd == "07") %>%
    group_by(customer_id) %>%
    summarise(sum_07 = sum(amount), .groups = "drop") %>%
    inner_join(df_receipt, by="customer_id") %>%
    group_by(customer_id) %>%
    summarise(sum_all=sum(amount), 
              sum_07=max(sum_07), .groups = "drop") %>%
    mutate(sales_rate = sum_07 / sum_all) %>%
    slice(1:10)

customer_id    sum_all sum_07 sales_rate
1  CS001113000004 1298    1298   1.0000000 
2  CS001114000005  626     486   0.7763578 
3  CS001115000010 3044    2694   0.8850197 
4  CS001205000004 1988     346   0.1740443 
5  CS001205000006 3337    2004   0.6005394 
6  CS001212000027  448     200   0.4464286 
7  CS001212000031  296     296   1.0000000 
8  CS001212000046  228     108   0.4736842 
9  CS001212000070  456     308   0.6754386 
10 CS001213000018  243     145   0.5967078

---
> R-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [78]:
df_receipt[c("customer_id", "sales_ymd")] %>%
    distinct(.,.keep_all=TRUE) %>%
    inner_join(df_customer[c("customer_id","application_date")], 
                            by="customer_id") %>%
    mutate(elapsed_days = as.integer(
                        strptime(as.character(sales_ymd), "%Y%m%d")
                                - strptime(application_date, "%Y%m%d"))) %>%
    select(customer_id, sales_ymd, application_date, elapsed_days) %>%
    slice(1:10)

customer_id    sales_ymd application_date elapsed_days
1  CS006214000001 20181103  20150201         1371        
2  CS008415000097 20181118  20150322         1337        
3  CS028414000014 20170712  20150711          732        
4  CS025415000050 20180821  20160131          933        
5  CS003515000195 20190605  20150306         1552        
6  CS024514000042 20181205  20151010         1152        
7  CS040415000178 20190922  20150627         1548        
8  CS027514000015 20191010  20151101         1439        
9  CS025415000134 20190918  20150720         1521        
10 CS021515000126 20171010  20150508          886

---
> R-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [79]:
df_receipt[c("customer_id", "sales_ymd")] %>%
    distinct(., .keep_all = TRUE) %>%
    inner_join(df_customer[c("customer_id", "application_date")], by = "customer_id") %>%
    mutate(elapsed_months = 
           trunc(time_length(
               interval(
                   strptime(application_date, "%Y%m%d"),
                   strptime(as.character(sales_ymd), "%Y%m%d")
               ), "month"))) %>%
    select(customer_id, sales_ymd, application_date, elapsed_months) %>%
    slice(1:10)

customer_id    sales_ymd application_date elapsed_months
1  CS006214000001 20181103  20150201         45            
2  CS008415000097 20181118  20150322         43            
3  CS028414000014 20170712  20150711         24            
4  CS025415000050 20180821  20160131         30            
5  CS003515000195 20190605  20150306         50            
6  CS024514000042 20181205  20151010         37            
7  CS040415000178 20190922  20150627         50            
8  CS027514000015 20191010  20151101         47            
9  CS025415000134 20190918  20150720         49            
10 CS021515000126 20171010  20150508         29

---
> R-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [80]:
df_receipt[c("customer_id", "sales_ymd")] %>%
    distinct(., .keep_all = TRUE) %>%
    inner_join(df_customer[c("customer_id", "application_date")], by = "customer_id") %>%
    mutate(elapsed_years = 
           trunc(time_length(interval(
               strptime(application_date, "%Y%m%d"),
               strptime(as.character(sales_ymd), "%Y%m%d")), "year")))%>%
    select(customer_id, sales_ymd, application_date, elapsed_years) %>%
    slice(1:10)

customer_id    sales_ymd application_date elapsed_years
1  CS006214000001 20181103  20150201         3            
2  CS008415000097 20181118  20150322         3            
3  CS028414000014 20170712  20150711         2            
4  CS025415000050 20180821  20160131         2            
5  CS003515000195 20190605  20150306         4            
6  CS024514000042 20181205  20151010         3            
7  CS040415000178 20190922  20150627         4            
8  CS027514000015 20191010  20151101         3            
9  CS025415000134 20190918  20150720         4            
10 CS021515000126 20171010  20150508         2

---
> R-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [81]:
df_receipt[c("customer_id", "sales_ymd")] %>%
    distinct(., .keep_all = TRUE) %>%
    inner_join(df_customer[c("customer_id","application_date")],
               by="customer_id") %>%
    mutate(elapsed_epoch = 
           as.numeric(strptime(as.character(sales_ymd), "%Y%m%d")) 
            - as.numeric(strptime(application_date, "%Y%m%d"))) %>%
    select(customer_id, sales_ymd, application_date, elapsed_epoch) %>%
    slice(1:10)

customer_id    sales_ymd application_date elapsed_epoch
1  CS006214000001 20181103  20150201         118454400    
2  CS008415000097 20181118  20150322         115516800    
3  CS028414000014 20170712  20150711          63244800    
4  CS025415000050 20180821  20160131          80611200    
5  CS003515000195 20190605  20150306         134092800    
6  CS024514000042 20181205  20151010          99532800    
7  CS040415000178 20190922  20150627         133747200    
8  CS027514000015 20191010  20151101         124329600    
9  CS025415000134 20190918  20150720         131414400    
10 CS021515000126 20171010  20150508          76550400

---
> R-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [82]:
df_receipt["sales_ymd"] %>%
    # 以下では開始日が日曜日となるため1日前シフトさせることで日曜日を前週の最終日に変換している。
    # その状態でfloor_dateをすると、週の開始日として前週の日曜日が取得できるため、
    # 開始日を月曜日にするために１を足している。
    mutate(monday = as.Date(floor_date(
        strptime(as.character(sales_ymd), "%Y%m%d") - 1 , 
                    unit = "week")) + 1) %>%
    mutate(elapsed_days = as.integer(
           as.Date(
               strptime(as.character(sales_ymd), "%Y%m%d")) - monday)) %>%
    select(sales_ymd, elapsed_days, monday) %>%
    slice(1:10)

sales_ymd elapsed_days monday    
1  20181103  5            2018-10-29
2  20181118  6            2018-11-12
3  20170712  2            2017-07-10
4  20190205  1            2019-02-04
5  20180821  1            2018-08-20
6  20190605  2            2019-06-03
7  20181205  2            2018-12-03
8  20190922  6            2019-09-16
9  20170504  3            2017-05-01
10 20191010  3            2019-10-07

---
> R-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [83]:
head(sample_frac(tbl = df_customer, size = 0.01), 10)

customer_id    customer_name gender_cd gender birth_day  age postal_cd
1  CS038514000102 藤島 里穂     1         女性   1960-03-12 59  279-0043 
2  CS049403000002 筒井 礼子     9         不明   1977-09-27 41  144-0044 
3  CS002514000317 玉置 璃奈子   1         女性   1959-12-02 59  185-0014 
4  CS022512000181 河内 倫子     1         女性   1967-02-18 52  240-0113 
5  CS019413000146 杉野 咲       1         女性   1974-02-10 45  176-0006 
6  CS003313000530 豊田 恵梨香   1         女性   1984-03-17 35  182-0006 
7  CS001413000361 上杉 花       1         女性   1974-08-24 44  144-0054 
8  CS024514000128 岡本 夏空     1         女性   1965-06-11 53  214-0036 
9  CS021515000244 黒谷 みゆき   1         女性   1966-11-05 52  259-1132 
10 CS012404000008 松谷 翔太     0         男性   1975-02-02 44  231-0827 
   address                              application_store_cd application_date
1  千葉県浦安市富士見**********         S13038               20150603        
2  東京都大田区本羽田**********         S14049               20161223        
3  東京都国分寺市東恋ケ窪**********     S13002               20170425        
4  神奈川県三浦郡葉山町長柄**********   S14022               20161119        
5  東京都練馬区栄町**********           S13019               20141119        
6  東京都調布市西つつじケ丘**********   S13003               20170316        
7  東京都大田区新蒲田**********         S13001               20160319        
8  神奈川県川崎市多摩区南生田********** S14024               20171023        
9  神奈川県伊勢原市桜台**********       S14021               20160511        
10 神奈川県横浜市中区本牧和田********** S14012               20150821        
   status_cd   
1  8-20100426-9
2  0-00000000-0
3  0-00000000-0
4  0-00000000-0
5  0-00000000-0
6  0-00000000-0
7  0-00000000-0
8  0-00000000-0
9  7-20101014-A
10 0-00000000-0

---
> R-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [84]:
set.seed(71)

df_customer %>%
    group_by(gender_cd) %>%
    sample_frac(0.1) %>%
    summarise(customer_num = n(), .groups = "drop")

gender_cd customer_num
1 0          298        
2 1         1792        
3 9          107

---
> R-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [85]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    mutate(log_sum_amount = log(sum_amount + 0.5)) %>%
    filter(abs(log_sum_amount - mean(log_sum_amount)) 
                                / sd(log_sum_amount) > 3) %>%
    slice(1:10)

customer_id    sum_amount log_sum_amount
1 ZZ000000000000 12395003   16.3328

---
> R-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [86]:
df_receipt %>%
    group_by(customer_id) %>%
    filter(!grepl("^Z", customer_id)) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    filter(
        sum_amount < quantile(sum_amount)[2] 
                     - 1.5 * (quantile(sum_amount)[4] - quantile(sum_amount)[2])
        |
        sum_amount > quantile(sum_amount)[4] 
                     + 1.5 * (quantile(sum_amount)[4] - quantile(sum_amount)[2])
    ) %>%
    slice(1:10)

customer_id    sum_amount
1  CS001414000048  8584     
2  CS001605000009 18925     
3  CS002415000594  9568     
4  CS004414000181  9584     
5  CS005415000137  8734     
6  CS006414000001  9156     
7  CS006414000029  9179     
8  CS006415000105 10042     
9  CS006415000147 12723     
10 CS006415000157 10648

---
> R-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [87]:
sapply(df_product, function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 7                  7

---
> R-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [88]:
df_product_1 <- na.omit(df_product)

paste("削除前:", nrow(df_product))
paste("削除後:", nrow(df_product_1))

[1] "削除前: 10030"

[1] "削除後: 10023"

---
> R-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [89]:
price_mean <- round(mean(df_product$unit_price, na.rm = TRUE))

cost_mean <- round(mean(df_product$unit_cost, na.rm = TRUE))

df_product_2 <- df_product %>%
    replace_na(list(unit_price = price_mean, unit_cost = cost_mean))
    
sapply(df_product_2, function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 0                  0

---
> R-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [90]:
price_median <- round(median(df_product$unit_price, na.rm = TRUE))

cost_median <- round(median(df_product$unit_cost, na.rm = TRUE))

df_product_3 <- df_product %>%
    replace_na(list(unit_price = price_median, unit_cost = cost_median))
    
sapply(df_product_3, function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 0                  0

---
> R-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [91]:
df_product_4 <- df_product %>%
    group_by(category_small_cd) %>%
    summarise(price_median = round(median(unit_price, na.rm = TRUE)), 
              cost_median = round(median(unit_cost, na.rm = TRUE)),
              .groups = "drop") %>%
    inner_join(df_product, by = "category_small_cd") %>%
    mutate(unit_price = ifelse(is.na(unit_price), price_median, unit_price),
           unit_cost = ifelse(is.na(unit_cost), cost_median, unit_cost))

sapply(df_product_4, function(x) sum(is.na(x)))

category_small_cd       price_median        cost_median         product_cd 
                 0                  0                  0                  0 
 category_major_cd category_medium_cd         unit_price          unit_cost 
                 0                  0                  0                  0

---
> R-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [92]:
df_receipt_2019 <- df_receipt %>%
                    filter(20190101 <= sales_ymd & sales_ymd <= 20191231) %>%
                    group_by(customer_id) %>%
                    summarise(amount_2019 = sum(amount), .groups = "drop")

df_receipt_all <- df_receipt %>%
                    group_by(customer_id) %>%
                    summarise(amount_all = sum(amount), .groups = "drop")

df_sales_rate <- left_join(df_customer["customer_id"],
                           df_receipt_2019, by = "customer_id") %>%
                    left_join(df_receipt_all, by = "customer_id") %>%
                    replace_na(list(amount_2019 = 0, amount_all = 0)) %>%
                    mutate(amount_rate = ifelse(amount_all == 0, 0,
                                                amount_2019 / amount_all))

df_sales_rate %>%
    filter(amount_rate > 0) %>%
    slice(1:10)

customer_id    amount_2019 amount_all amount_rate
1  CS031415000172 2971        5088       0.58392296 
2  CS015414000103  874        3122       0.27994875 
3  CS011215000048  248        3444       0.07200929 
4  CS029415000023 3767        5167       0.72904974 
5  CS035415000029 5823        7504       0.77598614 
6  CS023513000066  208         771       0.26977951 
7  CS035513000134  463        1565       0.29584665 
8  CS001515000263  216         216       1.00000000 
9  CS006415000279  229         229       1.00000000 
10 CS031415000106  215        7741       0.02777419

In [93]:
sapply(df_receipt_2019, function(x) sum(is.na(x)))

customer_id amount_2019 
          0           0

---
> R-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [94]:
df_geocode_1 <- df_geocode[c("postal_cd", "longitude" ,"latitude")] %>%
                group_by(postal_cd) %>%
                summarise(m_longiture = mean(longitude),
                          m_latitude = mean(latitude), .groups = "drop")

df_customer_1 <- inner_join(df_customer, df_geocode_1, by = "postal_cd")

head(df_customer_1, 10)

customer_id    customer_name gender_cd gender birth_day  age postal_cd
1  CS021313000114 大野 あや子   1         女性   1981-04-29 37  259-1113 
2  CS037613000071 六角 雅彦     9         不明   1952-04-01 66  136-0076 
3  CS031415000172 宇多田 貴美子 1         女性   1976-10-04 42  151-0053 
4  CS028811000001 堀井 かおり   1         女性   1933-03-27 86  245-0016 
5  CS001215000145 田崎 美紀     1         女性   1995-03-29 24  144-0055 
6  CS020401000016 宮下 達士     0         男性   1974-09-15 44  174-0065 
7  CS015414000103 奥野 陽子     1         女性   1977-08-09 41  136-0073 
8  CS029403000008 釈 人志       0         男性   1973-08-17 45  279-0003 
9  CS015804000004 松谷 米蔵     0         男性   1931-05-02 87  136-0073 
10 CS033513000180 安斎 遥       1         女性   1962-07-11 56  241-0823 
   address                            application_store_cd application_date
1  神奈川県伊勢原市粟窪**********     S14021               20150905        
2  東京都江東区南砂**********         S13037               20150414        
3  東京都渋谷区代々木**********       S13031               20150529        
4  神奈川県横浜市泉区和泉町********** S14028               20160115        
5  東京都大田区仲六郷**********       S13001               20170605        
6  東京都板橋区若木**********         S13020               20150225        
7  東京都江東区北砂**********         S13015               20150722        
8  千葉県浦安市海楽**********         S12029               20150515        
9  東京都江東区北砂**********         S13015               20150607        
10 神奈川県横浜市旭区善部町********** S14033               20150728        
   status_cd    m_longiture m_latitude
1  0-00000000-0 139.3178    35.41358  
2  0-00000000-0 139.8350    35.67193  
3  D-20100325-C 139.6897    35.67374  
4  0-00000000-0 139.4836    35.39125  
5  6-20090929-2 139.7078    35.54084  
6  0-00000000-0 139.6724    35.77073  
7  B-20100609-B 139.8360    35.67818  
8  0-00000000-0 139.9047    35.65422  
9  0-00000000-0 139.8360    35.67818  
10 6-20080506-5 139.5146    35.45013

---
> R-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

In [95]:
calc_distance <- function(x1, y1, x2, y2) {
    distance <- 6371 * acos(
                        sin(x1 * pi / 180) 
                        * sin(x2 * pi / 180) 
                        + cos(x1 * pi / 180) 
                        * cos(x2 * pi / 180) 
                        * cos((y1 * pi / 180) - (y2 * pi / 180 )))
    print(class(distance)) # 念の為 計算結果のclassを確認
    return(distance)
}

inner_join(df_customer_1, df_store, 
           by = c("application_store_cd" = "store_cd")) %>%
    rename(customer_address = address.x, store_address = address.y) %>%
    mutate(distance = 
           calc_distance(m_latitude, m_longiture, latitude, longitude)) %>%
    select(customer_id, customer_address, store_address, distance) %>%
    slice(1:10)

[1] "numeric"


customer_id    customer_address                  
1  CS021313000114 神奈川県伊勢原市粟窪**********    
2  CS037613000071 東京都江東区南砂**********        
3  CS031415000172 東京都渋谷区代々木**********      
4  CS028811000001 神奈川県横浜市泉区和泉町**********
5  CS001215000145 東京都大田区仲六郷**********      
6  CS020401000016 東京都板橋区若木**********        
7  CS015414000103 東京都江東区北砂**********        
8  CS029403000008 千葉県浦安市海楽**********        
9  CS015804000004 東京都江東区北砂**********        
10 CS033513000180 神奈川県横浜市旭区善部町**********
   store_address                      distance 
1  神奈川県伊勢原市伊勢原四丁目       1.3944087
2  東京都江東区南砂一丁目             1.4511822
3  東京都渋谷区初台二丁目             0.4117335
4  神奈川県横浜市瀬谷区二ツ橋町       8.0651960
5  東京都大田区仲六郷二丁目           1.2684210
6  東京都北区十条仲原三丁目           4.1859046
7  東京都江東区南砂二丁目             1.4496734
8  千葉県浦安市東野一丁目             0.8048581
9  東京都江東区南砂二丁目             1.4496734
10 神奈川県横浜市瀬谷区阿久和西一丁目 1.9569470

---
> R-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [96]:
df_sales_amount <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

df_customer_u <- left_join(df_customer, df_sales_amount, by = "customer_id") %>%
    mutate(sum_amount = ifelse(is.na(sum_amount), 0, sum_amount)) %>%
    arrange(desc(sum_amount), customer_id) %>%
    distinct(customer_name, postal_cd, .keep_all = TRUE) 

print(paste(
    "df_customer_cnt:", nrow(df_customer),
    "df_customer_u_cnt:", nrow(df_customer_u),
    "diff:", nrow(df_customer) - nrow(df_customer_u)))

[1] "df_customer_cnt: 21971 df_customer_u_cnt: 21941 diff: 30"


---
> R-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [97]:
df_customer_n = inner_join(df_customer, 
                           df_customer_u[c("customer_id", 
                                           "customer_name", 
                                           "postal_cd")],
                           by = c("customer_name", "postal_cd")) %>% 
                    rename(customer_id = customer_id.x, 
                           integration_id = customer_id.y)

customer_id_cnt <- length(unique(df_customer_n$customer_id))
integration_id_cnt <- length(unique(df_customer_n$integration_id))

print(paste("ID数の差", customer_id_cnt - integration_id_cnt))

[1] "ID数の差 30"


---
> R-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [98]:
set.seed(71)

#rsampleのinitial_splitを使った例
df_sales_customer <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    filter(sum_amount > 0)

df_tmp <- inner_join(df_customer, df_sales_customer, by = "customer_id")

split <- initial_split(df_tmp, prop = 0.8)

df_customer_train <- training(split)
df_customer_test <- testing(split)

# 確認
print(paste("学習データ割合:", nrow(df_customer_train) / nrow(df_tmp)))
print(paste("テストデータ割合:", nrow(df_customer_test) / nrow(df_tmp)))

[1] "学習データ割合: 0.799903684083795"
[1] "テストデータ割合: 0.200096315916205"


---
> R-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [99]:
#caretのcreateTimeSlicesを使った例
df_ts_amount <- df_receipt %>%
    group_by(substr(as.character(sales_ymd), 1, 6)) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

colnames(df_ts_amount) <- c("sales_ym", "sales_amount")

timeSlice <- createTimeSlices(df_ts_amount$sales_ym, 
                              initialWindow = 12, 
                              horizon = 6, 
                              skip = 5, 
                              fixedWindow = TRUE)

df_train_1 <- df_ts_amount[timeSlice$train[[1]],]
df_train_2 <- df_ts_amount[timeSlice$train[[2]],]
df_train_3 <- df_ts_amount[timeSlice$train[[3]],]

df_test_1 <- df_ts_amount[timeSlice$test[[1]],]
df_test_2 <- df_ts_amount[timeSlice$test[[2]],]
df_test_3 <- df_ts_amount[timeSlice$test[[3]],]

In [100]:
# df_train_2とdf_train_3の表示は割愛
df_train_1

sales_ym sales_amount
1  201701   902056      
2  201702   764413      
3  201703   962945      
4  201704   847566      
5  201705   884010      
6  201706   894242      
7  201707   959205      
8  201708   954836      
9  201709   902037      
10 201710   905739      
11 201711   932157      
12 201712   939654

In [101]:
# df_test_2とdf_test_３の表示は割愛
df_test_1

sales_ym sales_amount
1 201801    944509     
2 201802    864128     
3 201803    946588     
4 201804    937099     
5 201805   1004438     
6 201806   1012329

---
> R-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [102]:
#recipesパッケージを使った例
df_sales_amount <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    right_join(df_customer, by = "customer_id") %>%
    mutate(is_buy_flag = factor(ifelse(is.na(sum_amount), 0, 1)))

df_down_sampling <- df_sales_amount %>% 
    recipe() %>% 
    step_downsample(is_buy_flag, seed = 71) %>%
    prep() %>% 
    juice()

df_down_sampling %>%
    group_by(is_buy_flag) %>%
    summarise(cnt = n(), .groups = "drop")

is_buy_flag cnt 
1 0           8306
2 1           8306

---
> R-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

In [103]:
df_gender_std = unique(df_customer[c("gender_cd", "gender")])

df_customer_std = df_customer[, colnames(df_customer) != "gender"]

In [104]:
# データの内容確認
head(df_customer_std, n = 3)

customer_id    customer_name gender_cd birth_day  age postal_cd
1 CS021313000114 大野 あや子   1         1981-04-29 37  259-1113 
2 CS037613000071 六角 雅彦     9         1952-04-01 66  136-0076 
3 CS031415000172 宇多田 貴美子 1         1976-10-04 42  151-0053 
  address                        application_store_cd application_date
1 神奈川県伊勢原市粟窪********** S14021               20150905        
2 東京都江東区南砂**********     S13037               20150414        
3 東京都渋谷区代々木**********   S13031               20150529        
  status_cd   
1 0-00000000-0
2 0-00000000-0
3 D-20100325-C

In [105]:
# データの内容確認
head(df_gender_std, n = 3)

gender_cd gender
1 1         女性  
2 9         不明  
6 0         男性

---
> R-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [106]:
df_product_full <- inner_join(df_product, 
                              df_category[c("category_small_cd", 
                                            "category_major_name",
                                            "category_medium_name",
                                            "category_small_name")], 
                              by = "category_small_cd")

# データの内容確認
head(df_product_full, n = 3)

product_cd category_major_cd category_medium_cd category_small_cd unit_price
1 P040101001 04                0401               040101            198       
2 P040101002 04                0401               040101            218       
3 P040101003 04                0401               040101            230       
  unit_cost category_major_name category_medium_name category_small_name
1 149       惣菜                御飯類               弁当類             
2 164       惣菜                御飯類               弁当類             
3 173       惣菜                御飯類               弁当類

---
> R-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [107]:
# 解答ファイルの置き場所が設問ファイルと異なるため、パスが'../data'となっている点に注意
write.csv(df_product_full, "../data/R_df_product_full_UTF-8_header.csv", 
          row.names=FALSE,
          fileEncoding = "UTF-8")

---
> R-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [108]:
# 解答ファイルの置き場所が設問ファイルと異なるため、パスが'../data'となっている点に注意
write.csv(df_product_full, "../data/R_df_product_full_CP932_header.csv", 
          row.names = FALSE,
          fileEncoding = "CP932")

---
> R-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [109]:
# 解答ファイルの置き場所が設問ファイルと異なるため、パスが'../data'となっている点に注意
write.table(df_product_full, "../data/R_df_product_full_UTF-8_noh.csv", 
            row.names = FALSE,
            col.names = FALSE, sep = ",", fileEncoding = "UTF-8")

---
> R-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [110]:
# 解答ファイルの置き場所が設問ファイルと異なるため、パスが'../data'となっている点に注意
c_class <- c(NA, "character", "character", "character", NA, NA, NA, NA, NA)

df_product_full <- read.csv("../data/R_df_product_full_UTF-8_header.csv", 
                           colClasses = c_class,
                           fileEncoding = "UTF-8")

head(df_product_full, 3)

product_cd category_major_cd category_medium_cd category_small_cd unit_price
1 P040101001 04                0401               040101            198       
2 P040101002 04                0401               040101            218       
3 P040101003 04                0401               040101            230       
  unit_cost category_major_name category_medium_name category_small_name
1 149       惣菜                御飯類               弁当類             
2 164       惣菜                御飯類               弁当類             
3 173       惣菜                御飯類               弁当類

---
> R-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [111]:
# コード例1（後から項目名をつける）
# 解答ファイルの置き場所が設問ファイルと異なるため、パスが'../data'となっている点に注意
c_class <- c(NA, "character", "character", "character", NA, NA, NA, NA, NA)

df_product_full <- read.csv("../data/R_df_product_full_UTF-8_noh.csv", 
                           colClasses = c_class,
                           header = FALSE, fileEncoding = "UTF-8")

colnames(df_product_full) <- c("product_cd", "category_major_cd",
                               "category_medium_cd", "category_small_cd",
                               "unit_price", "unit_cost", 
                               "category_major_name", "category_medium_name", 
                               "category_small_name")

head(df_product_full, 3)

product_cd category_major_cd category_medium_cd category_small_cd unit_price
1 P040101001 04                0401               040101            198       
2 P040101002 04                0401               040101            218       
3 P040101003 04                0401               040101            230       
  unit_cost category_major_name category_medium_name category_small_name
1 149       惣菜                御飯類               弁当類             
2 164       惣菜                御飯類               弁当類             
3 173       惣菜                御飯類               弁当類

In [112]:
# コード例2（先に項目名を定義する）
# 解答ファイルの置き場所が設問ファイルと異なるため、パスが'../data'となっている点に注意
c_names <- c("product_cd", "category_major_cd", "category_medium_cd",
             "category_small_cd", "unit_price","unit_cost",
             "category_major_name", "category_medium_name",
             "category_small_name")

c_class <- c(NA, "character", "character", "character", NA, NA, NA, NA, NA)

df_product_full <- read.csv("../data/R_df_product_full_UTF-8_noh.csv", 
                           col.names = c_names,
                           colClasses = c_class,
                           header = FALSE, fileEncoding = "UTF-8")

head(df_product_full, 3)

product_cd category_major_cd category_medium_cd category_small_cd unit_price
1 P040101001 04                0401               040101            198       
2 P040101002 04                0401               040101            218       
3 P040101003 04                0401               040101            230       
  unit_cost category_major_name category_medium_name category_small_name
1 149       惣菜                御飯類               弁当類             
2 164       惣菜                御飯類               弁当類             
3 173       惣菜                御飯類               弁当類

---
> R-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [113]:
# 解答ファイルの置き場所が設問ファイルと異なるため、パスが'../data'となっている点に注意
write.table(df_product_full, "../data/R_df_product_full_UTF-8_header.tsv", 
            row.names = FALSE, sep = "\t", 
            fileEncoding = "UTF-8")

---
> R-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [114]:
# 解答ファイルの置き場所が設問ファイルと異なるため、パスが'../data'となっている点に注意
c_class <- c(NA, "character", "character", "character", NA, NA, NA, NA, NA)

df_product_tmp <- read.table("../data/R_df_product_full_UTF-8_header.tsv", 
                             colClasses = c_class,
                             header = TRUE, fileEncoding = "UTF-8")

head(df_product_tmp,3)

product_cd category_major_cd category_medium_cd category_small_cd unit_price
1 P040101001 04                0401               040101            198       
2 P040101002 04                0401               040101            218       
3 P040101003 04                0401               040101            230       
  unit_cost category_major_name category_medium_name category_small_name
1 149       惣菜                御飯類               弁当類             
2 164       惣菜                御飯類               弁当類             
3 173       惣菜                御飯類               弁当類

# これで１００本終わりです。おつかれさまでした！